In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import torch
from random import randint
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from transformers import BertForSequenceClassification
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
from torch import Tensor
from torch.nn import Transformer
import math
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from tqdm.auto import tqdm
import csv
from nltk.translate.bleu_score import sentence_bleu
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import pandas as pd
trainFile = "/content/drive/MyDrive/NLP_Project/hindi_train_final.csv"
trainDF = pd.read_csv(trainFile, encoding = "utf-8")
valFile = "/content/drive/MyDrive/NLP_Project/hindi_val_final.csv"
valDF = pd.read_csv(valFile, encoding = "utf-8")
testFile = "/content/drive/MyDrive/NLP_Project/hindi_test_final.csv"
testDF = pd.read_csv(testFile, encoding = "utf-8")

In [4]:
trainID = list(trainDF['Id'])
trainHeading = list(trainDF['Heading'])
trainArticle = list(trainDF['Article'])
trainSummary = list(trainDF['Summary'])
valID = list(valDF['Id'])
valHeading = list(valDF['Heading'])
valArticle = list(valDF['Article'])
valSummary = list(valDF['Summary'])
testID = list(testDF['Id'])
testHeading = list(testDF['Heading'])
testArticle = list(testDF['Article'])
testSummary = list(testDF['Summary'])

print(len(trainArticle))
print(trainArticle[0])
print(len(valArticle))
print(valArticle[0])
print(len(valArticle))
print(valArticle[0])

19102
हिमाचल प्रदेश के चंबा जिले में सोमवार देर रात भूकंप के झटके महसूस किए गए। मौसम विज्ञान केंद्र शिमला के अनुसार, रात 10 बजकर 38 मिनट पर 2 बार झटके लगे। रिएक्टर स्केल पर इसकी तीव्रता 3.70 आंकी गई।
गनीमत यह रही कि इसमें किसी तरह का कोई जान व माल का नुकसान नहीं हुआ, लेकिन भूकंप का केंद्र भरमौर के साथ लगता क्षेत्र रहा। वहीं जमीन के अंदर इसकी गहराई 5 किलोमीटर तक रही।
भूकंप के झटके आसपास के भरमौर, किलाड़ व चंबा में भी महसूस किए गए। बताया जा रहा है कि 2 झटके लगने के बाद कुछ लोग अपने घरों से बाहर निकल आए थे।
सिस्मिक जोन-5 में आते हैं प्रदेश के कई क्षेत्र
बता दें कि हिमाचल के चंबा, कुल्लू, शिमला, किन्नौर और कांगड़ा के कई क्षेत्र भूकंप की दृष्टि से अति संवेदनशील सिस्मिक जोन-5 में आते हैं। यहां बार-बार भूकंप के झटके महसूस किए जाते हैं। तुर्की में कुछ दिन पहले आए भूकंप से तबाही के बाद प्रदेश के लोग डरे व सहमे हुए भी हैं।
1061
श्रीनगर: श्रीनगर शहर के बाहरी इलाके में स्थित परिमपोरा में पिछले सप्ताह सुरक्षा बलों के साथ मुठभेड़ में मारे गए तीन कथित आतंकवादियों के परिवारों ने सोमवार को उनके शव वापस क

In [5]:
def applyTfIdfVectorizer(article):
  # Obtaining the list of all sentences
  sentences = re.split(r'[?।]\s', article)
  sentences = [s.strip() for s in sentences if s.strip()]

  tfidfVectorizer = TfidfVectorizer()
  tfidfMatrix = tfidfVectorizer.fit_transform(sentences)
  #print(type(tfidfMatrix))
  sentenceScores = np.sum(tfidfMatrix, axis=1)
  #print(type(sentenceScores))
  sortedIndices = np.argsort(sentenceScores, axis=0)[::-1].flatten()
  #print(type(sortedIndices))
  #print(sortedIndices)
  sortedIndices = sortedIndices.tolist()
  sortedIndices = sortedIndices[0]
  #print(sortedIndices)
  rankedSentences = []
  finalScores = []
  for ind in sortedIndices:
    rankedSentences.append(sentences[ind])
    finalScores.append(sentenceScores[ind].tolist()[0][0])

  return rankedSentences, finalScores



In [6]:
senty, scorey = applyTfIdfVectorizer(trainArticle[0])
print(senty)
print(scorey)
print(len(senty))
print(len(scorey))

['मौसम विज्ञान केंद्र शिमला के अनुसार, रात 10 बजकर 38 मिनट पर 2 बार झटके लगे', 'गनीमत यह रही कि इसमें किसी तरह का कोई जान व माल का नुकसान नहीं हुआ, लेकिन भूकंप का केंद्र भरमौर के साथ लगता क्षेत्र रहा', 'बताया जा रहा है कि 2 झटके लगने के बाद कुछ लोग अपने घरों से बाहर निकल आए थे', 'सिस्मिक जोन-5 में आते हैं प्रदेश के कई क्षेत्र\nबता दें कि हिमाचल के चंबा, कुल्लू, शिमला, किन्नौर और कांगड़ा के कई क्षेत्र भूकंप की दृष्टि से अति संवेदनशील सिस्मिक जोन-5 में आते हैं', 'वहीं जमीन के अंदर इसकी गहराई 5 किलोमीटर तक रही', 'रिएक्टर स्केल पर इसकी तीव्रता 3.70 आंकी गई', 'तुर्की में कुछ दिन पहले आए भूकंप से तबाही के बाद प्रदेश के लोग डरे व सहमे हुए भी हैं।', 'हिमाचल प्रदेश के चंबा जिले में सोमवार देर रात भूकंप के झटके महसूस किए गए', 'भूकंप के झटके आसपास के भरमौर, किलाड़ व चंबा में भी महसूस किए गए', 'यहां बार-बार भूकंप के झटके महसूस किए जाते हैं']
[3.1333758291720746, 3.12060618876171, 2.9645472485643927, 2.9435932363394253, 2.8138470725473774, 2.6374758285825757, 2.435369425566895, 2.41988471850964, 2.20

In [11]:
extractedTrainArticles = []
cnt = 0
for article in trainArticle:
  cnt = cnt + 1
  if cnt==14892:
    extractedTrainArticles.append("")
    continue
  orderedSentences, orderedScores = applyTfIdfVectorizer(article)
  # sortedIndices = np.argsort(sentenceScores)[::-1]
  # sentenceScores.sort(reverse = True)
  # rankedSentences = []
  # for ind in sortedIndices:
  #   rankedSentences.append(allSentences[ind])

  extractedStr = ""
  numTokens = 0
  maxTokens = 256
  for senty in orderedSentences:
    numTokens = numTokens + len(senty.split())
    if numTokens>maxTokens:
      break
    extractedStr = extractedStr + senty + "। "

  extractedTrainArticles.append(extractedStr)
  print(cnt)

Streaming output truncated to the last 5000 lines.
14102
14103
14104
14105
14106
14107
14108
14109
14110
14111
14112
14113
14114
14115
14116
14117
14118
14119
14120
14121
14122
14123
14124
14125
14126
14127
14128
14129
14130
14131
14132
14133
14134
14135
14136
14137
14138
14139
14140
14141
14142
14143
14144
14145
14146
14147
14148
14149
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14174
14175
14176
14177
14178
14179
14180
14181
14182
14183
14184
14185
14186
14187
14188
14189
14190
14191
14192
14193
14194
14195
14196
14197
14198
14199
14200
14201
14202
14203
14204
14205
14206
14207
14208
14209
14210
14211
14212
14213
14214
14215
14216
14217
14218
14219
14220
14221
14222
14223
14224
14225
14226
14227
14228
14229
14230
14231
14232
14233
14234
14235
14236
14237
14238
14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
1

In [12]:
extractedValArticles = []
for article in valArticle:
  orderedSentences, orderedScores = applyTfIdfVectorizer(article)

  extractedStr = ""
  numTokens = 0
  maxTokens = 256
  for senty in orderedSentences:
    numTokens = numTokens + len(senty.split())
    if numTokens>maxTokens:
      break
    extractedStr = extractedStr + senty + "। "

  extractedValArticles.append(extractedStr)

In [13]:
extractedTestArticles = []
for article in testArticle:
  orderedSentences, orderedScores = applyTfIdfVectorizer(article)


  extractedStr = ""
  numTokens = 0
  maxTokens = 256
  for senty in orderedSentences:
    numTokens = numTokens + len(senty.split())
    if numTokens>maxTokens:
      break
    extractedStr = extractedStr + senty + "। "

  extractedTestArticles.append(extractedStr)

In [14]:
finalTrainDF = pd.DataFrame({'Id': trainID, 'Heading': trainHeading, 'Article': extractedTrainArticles, 'Summary': trainSummary})
finalValDF = pd.DataFrame({'Id': valID, 'Heading': valHeading, 'Article': extractedValArticles, 'Summary': valSummary})
finalTestDF = pd.DataFrame({'Id': testID, 'Heading': testHeading, 'Article': extractedTestArticles, 'Summary': testSummary})

outTrain = "/content/drive/MyDrive/NLP_Project/hindi_train_final_TFIDF.csv"
outVal = "/content/drive/MyDrive/NLP_Project/hindi_val_final_TFIDF.csv"
outTest = "/content/drive/MyDrive/NLP_Project/hindi_test_final_TFIDF.csv"

finalTrainDF.to_csv(outTrain, index = False)
finalValDF.to_csv(outVal, index = False)
finalTestDF.to_csv(outTest, index = False)